In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [3]:
import numpy as np
import pandas as pd
import mysql.connector
import logging

In [4]:
DATASET_PATH = '../../datasets/amazon-sneakers'

In [5]:
items      = pd.read_json(f'{DATASET_PATH}/items.json')
interactions = pd.read_json(f'{DATASET_PATH}/interactions.json')

In [6]:
items.head(5)

,id,title,description,variant,url,rating,reviews,price
0,B0B8V49Z1V,FLYFUPPY Women's Slip-on Canvas Comfort Fashio...,None,False,https://m.media-amazon.com/images/I/81FUVhjkB1...,4.6,16,0.0
1,B08TRDBGQF,Nike Women's Low-Top Sneakers,These shoes provide the user with great comfor...,False,https://m.media-amazon.com/images/I/61DOlkBbfJ...,2.7,7,31.0
2,B06XVY23RT,Reebok Women's Club C Sneaker,None,False,https://m.media-amazon.com/images/I/81B-Vy65uk...,4.6,16053,13.075
3,B0785RGRWX,Reebok Men's Classic Harman Run Sneaker,None,False,https://m.media-amazon.com/images/I/71FaYc6pSM...,4.6,6501,10.465
4,B0BDZV1LCS,Steve Madden Ganzo-R Sneaker,None,False,https://m.media-amazon.com/images/I/71C-5pfm1D...,5.0,1 rating,0.0


In [7]:
items['description'] = items['description'] + ' (Price: ' + items['price'].astype(str) + ')'

In [8]:
items['item_id'] = pd.factorize(items['id'])[0] +1
items = items[['item_id', 'id', 'title', 'description']]
items.head(5)

,item_id,id,title,description
0,1,B0B8V49Z1V,FLYFUPPY Women's Slip-on Canvas Comfort Fashio...,NaN
1,2,B08TRDBGQF,Nike Women's Low-Top Sneakers,These shoes provide the user with great comfor...
2,3,B06XVY23RT,Reebok Women's Club C Sneaker,NaN
3,4,B0785RGRWX,Reebok Men's Classic Harman Run Sneaker,NaN
4,5,B0BDZV1LCS,Steve Madden Ganzo-R Sneaker,NaN


In [9]:
interactions['user_id'] = pd.factorize(interactions['username'])[0] + 1

In [10]:
interactions.head()

,username,item_id,rating,message,user_id
0,B.Garcia87,B08FQYQYPM,4.0,I bought this as a cheap shoe for work. I work...,1
1,hannamand,B08FQYQYPM,5.0,I bought both the Blue and Black versions of t...,2
2,lisam72,B08FQYQYPM,4.0,They weren't quite what I thought they would b...,3
3,Leo Trotter,B08FQYQYPM,4.0,Product was exactly as described. Great runnin...,4
4,Scott l,B08FQYQYPM,4.0,They work,5


In [11]:
interactions = interactions.merge(items, left_on='item_id', right_on='id')[['item_id_y', 'rating', 'user_id']]
interactions = interactions.rename(columns={'item_id_y': 'item_id'})

In [12]:
items = items[['item_id', 'title', 'description']]

In [13]:
items

,item_id,title,description
0,1,FLYFUPPY Women's Slip-on Canvas Comfort Fashio...,NaN
1,2,Nike Women's Low-Top Sneakers,These shoes provide the user with great comfor...
2,3,Reebok Women's Club C Sneaker,NaN
3,4,Reebok Men's Classic Harman Run Sneaker,NaN
4,5,Steve Madden Ganzo-R Sneaker,NaN
...,...,...,...
1780,45,New Balance Women's 237 V1 Classic Sneaker,The running shoes of the 1970s might best be d...
1781,18,Niluber Women's Chunky Platform Dad Lace-Up Ca...,Product Attributes Product Volume：‎ 11.52 x 9....
1782,390,GOOBON Air Shoes for Women Tennis Sports Athle...,NaN
1783,103,Women's Walking Shoes Sock Sneakers Slip on Me...,NaN


In [14]:
interactions

,item_id,rating,user_id
0,168,4.0,1
1,168,5.0,2
2,168,4.0,3
3,168,4.0,4
4,168,4.0,5
...,...,...,...
162789,236,3.0,4196
162790,236,5.0,4197
162791,236,5.0,4198
162792,236,5.0,4199


In [15]:
connection = mysql.connector.connect(
  host     = "localhost",
  user     = "root",
  password = "1234"
)

In [16]:
def execute(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
    except Exception as error:
        print(error)
        print(f'Query: {query}')


def insert_item(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_item(id, name, description, image)
        VALUES (
            :ID,
            ":NAME",
            ":DESC",
            ":IMAGE"
        );    
        """.replace(':ID', str(row['id'])) \
            .replace(':NAME', str(row['name'])) \
            .replace(':DESC', str(row['description']).replace('"', '')) \
            .replace(':IMAGE', str(row['image']))
    execute(connection, query)
        
def insert_interaction(connection, row):
    query = """
        INSERT INTO 
            recsys.recsysweb_interaction(item_id, user_id, rating)
        VALUES (
            ":ITEM_ID",
            ":USER_ID",
            ":RATING"
        );    
        """.replace(':ITEM_ID', str(int(row['item_id']))) \
            .replace(':USER_ID', str(int(row['user_id']))) \
            .replace(':RATING', str(row['rating']))
    execute(connection, query)

In [17]:
execute(connection, 'ALTER TABLE recsys.recsysweb_item MODIFY id int(11) NOT NULL')
execute(connection, 'DELETE FROM recsys.recsysweb_item')

1451 (23000): Cannot delete or update a parent row: a foreign key constraint fails 
(`recsys`.`recsysweb_interaction`, CONSTRAINT 
`recsysweb_interaction_item_id_818b6e1a_fk_recsysweb_item_id` FOREIGN KEY (`item_id`) 
REFERENCES `recsysweb_item` (`id`))

Query: DELETE FROM recsys.recsysweb_item

In [18]:
for _, row in movies.iterrows():
    insert_item(connection, row)    

NameError: name 'movies' is not defined

In [ ]:
execute(connection, 'DELETE FROM recsys.recsysweb_interaction')

In [ ]:
for _, row in interactions.iterrows():
    insert_interaction(connection, row)

In [ ]:
connection.close()

In [ ]:
interactions.user_id.max()